# IbsensXMLstrip (IXMLS)
Release *1.0.25*

Mars 2022

av vit.ass. Loke Sjølie

In [1]:
version = "1.0.25"

## Parametre
!**VIKTIG**! Under har du parametre for konversjonen. Disse er prekonfigurert og fungerer bra, men om du vil endre noe i output bør du følge instruksene her før du endrer på selve koden. Alle definerte parametre er trygge. Unntatt his:hisdel. Bare la den være på 1 med mindre du fikser den selv.

### his:hisdel
tag_hisdel kontrollerer følgende undertagger: *overwritten*, *overstrike*, *diagonalOverstrike*. ~~Om tag_hisdel = 2 erstattes alle hisdel-taggene med markdown (~), om tag_hisdel = 1 slettes alle helt, og om tag_hisdel er 0 kontrolleres hver type av sin egen verdi. Dvs at GRUPPEN tar prioritet over individuell tagg.

In [2]:
# 0 gjør ingenting, 1 sletter taggen med innholdet HELT, 2 SKAL ETTERHVERT beholde innholdet med markdown (~~)
# Gruppekontroll
tag_hisdel = 1 # Denne bør være 1 med mindre du tester noe.

# Individuell kontroll - brukes kun dersom gruppekontroll er 0
# DISSE FUNGERER IKKE ENDA - BRUK GRUPPEKONTROLL 1 ELLER 2, 3 FOR Å DEAKTIVERE. IKKE 0
tag_overwritten = 1
tag_overstrike = 1
tag_diagonaloverstrike = 1

### Filformat
Parametre for å kontrollere hvilket format som skal produseres. Merk at programmet ikke sletter eksisterende filer. Anbefaler å slette alt i output-folderen før det kjøres.

In [3]:
filsuffiks = "_RENTEKST" # Teksten som skal opptre etter hvert filnavn. Anbefaler "_RENTEKST" på norsk.
filformat = ".txt" # Formatet på de bearbeidede filene. Anbefaler .txt eller .md.

### Metadatafiler
1 er ja, 0 er nei. Refererer til filene lagt under ../metadata/.

In [4]:
createMetadata = 1

### Tagger
Parametre for å kontrollere hvilke tagger som skal slettes helt *med* innhold.

In [5]:
tags_to_erase = ["fw","handnote","teiheader","lb","figure","pb","note","anchor","back"]

### Tilfeldig resterende tegn etter sletting
Dette gjelder ting som blir igjen etter sletting av typen: "()", ") ) ()", "- - - - o s v o s v". Ibsen strøk riktignok ikke over disse tegnene, og de er heller ikke forstyrrende for analyse. Det er en *viss* risiko for at dette kan slette feil ting. Sett 1 for å fjerne, 0 for å la de være.

**Tar bare "()" per nå.** Vurderes som >99% trygt - "()" er ikke noe man ser i naturlig språk.

In [6]:
purgeNotation = 1

### Forklarende print
Sett verbose=1 om du vil ha ekstra printfunksjoner som rapporterer diverse detaljer. Har ingenting å si for output.

In [7]:
verbose = 0

---
## Program
Programmet er bygd slik at det MÅ kjøres på nytt dersom det stopper under kjøring.

### Bibliotek

In [8]:
from bs4 import BeautifulSoup # Hent BeautifulSoup-modulen (https://www.crummy.com/software/BeautifulSoup/)
import re # Requests
import os # Filsystem for lasting og lagring
import glob
import codecs # Dekoder for unicode
import csv # CSV-leser for unicode
from bs4 import Comment # BS4-addon for å håndtere(slette) kommentarer <!-- X -->
from datetime import date # Dato..
today = date.today() # Sett dato i dag
today = today.strftime("%d/%m/%Y") # Formater dato

### Oversett UNICODE
Ja, selve koden refererer til "asciis" og "ascii". Det er unicode da. :)

In [9]:
file = open(r'config\unicode.csv') # Egendefinert fil fra DTD.
# Andre diverse entiteter som manglet er også oversatt.
csvreader = csv.reader(file) # Importer  filen over
asciis = [] # Array for unicode-greier (eks. \u0023)
codes = [] # Array for ting som &nbsp - "html"-koder
for row in csvreader: # For hver rad
    htmlcode = row[0] # Html-tagger (de som er synlige i teksten)
    unicode = row[1] # Unicode (eg \u0023)
    ASCII = codecs.decode(unicode, 'unicode_escape') # Gjør om til unicode
    asciis.append(ASCII) # Legg til i array
    codes.append(htmlcode)
    if verbose == 1:
        print(htmlcode,"=>",ASCII)

### Slett tagger vi ikke vil ha

In [10]:
if tag_hisdel == 1:
    tags_to_erase.append("his:hisdel")
elif tag_hisdel == 2:
    print("Alle <his:hisdel>-tagger blir erstattet med markdown: { ~~ [innhold] ~~ }")
    print("Obs, dette fungerer ikke enda. :/")
    # for tag in content.findAll('his:hisdel'):
        # tag.replaceWith("~~ %s ~~" % a.string)
elif tag_hisdel == 0:
    if tag_overwritten == 1:
        tags_to_erase.append("overwritten") 
        # content.findAll("his:hisdel", {"rend" : "overwritten"})
    elif tag_overwritten == 2:
        print("TBI: markdown around overwrittens")
    if tag_overstrike == 1:
        tags_to_erase.append("overstrike") 
        # content.findAll("his:hisdel", {"rend" : "overstrike"})
    elif tag_overstrike == 2:
        print("TBI: markdown around overstrikes")
    if tag_diagonalOverstrike == 1:
        tags_to_erase.append("diagonalOverstrike")
        # content.findAll("his:hisdel", {"rend" : "diagonalOverstrike"})
    elif tag_overstrike == 2:
        print("TBI: markdown around diagonal overstrikes")

### GLOB
Etablerer filstier, dvs gir programmet mål og metode for å finne kildefilene.

In [11]:
paths = []
for path in glob.glob('xml_src/**/*.xml', recursive=True):
    paths.append(path)
#paths er liste over stier til filer i SRC
errors = []
outputs = []
for path in paths:
    path = path.replace('xml_src','xml_output')
    outputs.append(path)
#outputs er liste over stier til nye filer

#### Logg: filstier

In [12]:
if verbose == 1:
    i = 0
    for path in paths:
        print(i,path)
        i+=1

### Programkjerne V3
Brukes fra betaversjon 0.3. Denne kjernen diskriminerer ikke mellom *n* antall front-, body eller andre diverse tagger og har krokfunksjonalitet for å legge \n eller annet før/etter diskré tekster, som kan bli aktuelt ved brev.

In [13]:
# ACTION
i = 0
x = 0
y = 0
for thing in paths: # For hver sti (med endestasjon fil) i listen over stier
    supralinears = [] # Liste over supertall
    infralinears = [] # Liste over infratall
    samleparanteser = [] # Liste over samleparanteser
    comments = [] # Liste over kommentarer
    tagswereremoved = 0
    asciisfixedup = 0
    texts = "Henter alle instanser av <text>"
    #if "ht" in thing: # Ekskluder ikke-hovedtekster
    #IF HER
    if "noter" not in thing: # Fortsett hvis "noter" IKKE er del av filens tittel.
        print(i+1,thing)
        # Filhåndtering I - Les og last xml-filer inn i minne
        with open(thing, "r", encoding="utf-8") as file: # Åpne filen med encoding UTF8
            content = file.readlines() # Les innholdet som linjer
            content = "".join(content) # Linjene blir kombinert i en variabel

        # Filhåndtering II - Lag ny sti
        frags = outputs[i].split('\\') # Splitt outputstrengen for sti
        if "xml" in frags[2]: # Hvis filen er 3 ned, altså vi finner xml-filen her (eks. xml_src\dikt\~.xml)
            dirName = frags[0]+"\\"+frags[1] # Dirname blir 2 dyp (xml_output\DIRNAME\)
            metaName = frags[2] # Trenger et navn på filen
        else: # OBS!! OBS!! Hvis filen IKKE er 3 ned ANTAR vi at den er 4 ned, slik som de(n) er i dramafolderen.
            metaName = frags[3] # Navn på filen
            dirName = frags[0]+"\\"+frags[1]+"\\"+frags[2] # Dirname blir 3 dyp (xml_output\drama\DIRNAME\)
            archDir = frags[0]+"\\"+frags[1] # FAILSAFE for DRAMA: Drama er ett hakk lengre ned i hierarkiet enn alt annet. Skap mappe for å ta drama
            #OBS - koden her ANTAR at senteret ALLTID følger samme praksis. Koden må endres om praksis endres.
            if not os.path.exists(archDir): # Hvis subfolder ikke finnes (eks.\drama\BE\)
                os.mkdir(archDir) # Skap den

        metaFolder = dirName+"\\metadata" # Vi forbereder en metadatafolder ett steg under stien vi lagde over
        metaName = metaName[:-4] # Strip . x m l. Bruker ikke outputs[i] fordi den gir hele stien, metaName er fragment
        metaFile = metaFolder+"\\"+metaName+"_METADATA.txt" # Metadatafilen skal få samme filnavn med _metadata-suffiks
        outputs[i] = outputs[i][:-4] # Strip 4 tegn fra slutten av hver streng for å få bort .xml (4. 3X 2M 1L)

        if not os.path.exists(dirName): # Hvis folderen ikke eksisterer
            os.mkdir(dirName) # Lag folderen
            os.mkdir(metaFolder) # Legg metadata-folder inne i den igjen
        name = outputs[i]+filsuffiks+filformat # Outputfil får navnet X_SANERT.txt

        # BeautifulSoup - her begynner vi å endre på XML-strukturen.
        # STRING-METODER FUNGERER IKKE HER!
        content = BeautifulSoup(content, from_encoding="UTF-8") # Content er content som bs4-objekt...
        target = content.body
        
        # Krok for å separere text-tagger om ønskelig.
        texts = content.findAll("text") # 0 er wrapper.
#        wrapper = texts[0]
        if len(texts) > 1: # Hvis det er MER enn én text-tagg
            print("** LOGG Filen har",len(texts)-1,"tekster **") # DEBUG print hvor mange det er 
#            textcount = 0
#            for text in texts: # For hver text...
#                textcount += 1
#            print(textcount)
          
        # Opprett metadata
        if createMetadata == 1:
            print("Henter metadata...")
            try:
                titlestatement = content.find("titlestmt") # Finn title statement
                titlestatement.extract() # Hent (og fjern) title statement-delen
                #print("> Titlestmt")
            except:
                print("X Ingen title statement")
            try:
                filedescription = content.find("filedesc") # Finn filbeskrivelsen
                filedescription.extract() # Hent (og fjern) filbeskrivelsen
            except:
                print("X Ingen filbeskrivelse")

        # Teksten (alt under <text>)
        print("Konverterer teksten...")
        # Slett tagger vi ikke skal beholde
        if tag_hisdel == 2:
            print("> Sletter tagger")
            z = 0
            try:
                for tag in content.findAll('his:hisdel'):
                    tag.replaceWith(" ~~ %s ~~ " % a.string)
                    z += 1
                if z > 0:
                    print("> Erstattet",z,"hisdel-tagger med markdown-tagger.")
                else:
                    print("> his:hisdel-tagger er ikke 0, men 0 ble fjernet...? Hva.")
            except:
                print("> Ingen hisdel-tagger å endre.")

       # try:
        for tag in tags_to_erase: # For hver tagg
            to_remove = content.find_all(tag) # Finn alle instanser av taggen
            if len(to_remove) > 0: # Hvis det er 1 eller flere av taggen
                for remove in to_remove: # For hver av disse
                    remove.decompose() # Fjern taggen og innholdet helt
                    tagswereremoved += 1
        if tagswereremoved > 0:
            print(">",tagswereremoved,"uønskede tagger")
        supralinears = content.findAll(attrs={"rend": "supralinear", "subtype":"no"}) # Finn ^x-notasjon...
        infralinears = content.findAll(attrs={"rend": "infralinear", "subtype":"no"}) # Finn ^x-notasjon...
        samleparanteser = content.findAll(attrs={"rend":"bracedLeft"}) # Finn {-notasjon...
        if len(supralinears) > 0:
            for no in supralinears: # For hver av disse
                no.decompose() # Slett dem helt
            print(">",len(supralinears),"superlineære tallnoter")   
        if len(infralinears) > 0:
            for no in infralinears: # For hver av disse
                no.decompose() # Slett dem helt
            print(">",len(infralinears),"infralineære tallnoter")
        if len(samleparanteser) > 0:
            for samleparantes in samleparanteser:
                samleparantes.decompose()
            print(">",len(samleparanteser),"samleparanteser")

        comments = content.findAll(string=lambda text: isinstance(text, Comment))
        if len(comments) > 0:
            print(">",len(comments),"HTML-kommentarer")
            for c in comments:
                c.extract()
                
        content = str(content) # CONTENT ER NÅ STRING - STRINGMETODER FUNGERER, BS4-METODER FUNGERER IKKE
        #content = re.sub("(<!--.*?-->)", "", content, flags=re.DOTALL) # RegEx for kommentarutrydding
        z = 0 # Tell fra 0 i z
        for code in codes: # For hver kode
            if code in content: # Hvis koden finnes i teksten
                content = content.replace(code,asciis[z]) # Erstatt koden med tekst
                asciisfixedup += 1
                if verbose == 1:
                    print('Erstattet',code,'med',asciis[z]) # Print bekreftelse
            z+=1 # Neste kode etter å ha sjekket hver kode
        print(">",asciisfixedup,"unike unicode-tegn")
        
        # Tekstfunksjonene under er et salig rot og sakker sikkert koden noe helt grusomt.
        # Men det funker. ;)
        content = content.replace('<cell role="data">'," ") # Quickfix for å gjøre tables mindre rotete
        content = re.sub('<[^<]+>', "", content) # Slett alle resterende tagger UTEN å røre innholdet
        content = re.sub('\s+',' ', content) # Korriger alle instanser av multimellomrom til enkeltmellomrom
        content = content.replace("&typHyp; ","")
        content = content.replace("&typHyp;","")
        content = content.replace("\n"," ") # Slett newline og erstatt med mellomrom 
        content = content.replace("'“'","")
        content = content.replace("&dash;","-")
        content = content.replace("&"," & ")
        content = content.replace("& &"," & ")
        content = content.replace(". &",".")
        content = content.replace("'\"'", "")
        content = content.replace(" < ","")
        
        if purgeNotation == 1:
            content = content.replace("()","")
        content = re.sub('\s+',' ', content) # Korriger alle instanser av multimellomrom til enkeltmellomrom IGJEN.

        if tag_hisdel == 1: # Hvis vi sletter hisdel-taggen med innhold
            content = content.replace(" . ", " ")
            content = content.replace(" < ", "")
        content = content.strip()
        print("Lagrer...")
        
        # Filhåndtering III - Skriv til disk
        try: # Prøv å skrive til disk
            f = open(name, "w", encoding="utf-8") # Ikke glem encoding="utf-8", sånn som jeg gjorde.
            f.write(content) # Skriv innholdet i content-variabelen
            f.close()
            print(">> Opprettet",name,"i",dirName)
            x+=1
        except: # Hvis vi møter på feil, rapporter kilden
            print("XXX FILFEIL!",thing)
            f = open(name+"_ERROR", "w", encoding="utf-8")
            f.write("CRITICAL ERROR DURING WRITE SEQUENCE\nCHECK ENCODING")
            f.close()
            errors.append(name)
            y+=1
            #raise KeyboardInterrupt
        if createMetadata == 1:
            metadata = str(filedescription)+str(titlestatement)
            metadata = metadata.replace('<title level="a" type="main">',"Primary title: ")
            metadata = metadata.replace('<title level="s" type="main">',"Primary subtitle: ")
            metadata = metadata.replace('<title level="s" type="sub">',"Subtitle: ")
            metadata = metadata.replace("<author>","Author: ")
            metadata = metadata.replace('<title type="main">',"Title: ")
            metadata = re.sub('<[^<]+>', " ", metadata) # Fjern tagger
            metadata = re.sub('\s+',' ', metadata) # Korriger alle instanser av multimellomrom til enkeltmellomrom
            dateassign = "Converted to plain text from TEI-style XML",today,"with IbsensXMLStrip, version",version+"."
            decl = " ".join(dateassign)
            metadata += decl
            try: # Prøv å skrive metadata! til disk
                f = open(metaFile, "w", encoding="utf-8") # Ikke glem encoding="utf-8", sånn som jeg gjorde.
                f.write(metadata) # Skriv innholdet i metadata-variabelen
                f.close()
                print(">>> Metadata opprettet:",metaFile)
                x+=1
            except: # Hvis vi møter på feil, rapporter kilden
                print("YYY METADATA FILFEIL!",thing)
                f = open(metaFile+"_ERROR", "w", encoding="utf-8")
                f.write("CRITICAL ERROR DURING WRITE SEQUENCE\nCHECK ENCODING")
                f.close()
                errors.append(name)
                y+=1
                #raise KeyboardInterrupt
        print("\n")
        #IF HER
    #else:
     #   print("Skipped",thing)
    i+=1
percA = (i/len(paths))*100

1 xml_src\brev\B1844-1871ht.xml


C:\Users\lokes\Anaconda3\lib\site-packages\bs4\__init__.py:221: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


** LOGG Filen har 333 tekster **
Henter metadata...
Konverterer teksten...
> 4563 uønskede tagger
> 1 HTML-kommentarer
> 13 unike unicode-tegn
Lagrer...
>> Opprettet xml_output\brev\B1844-1871ht_RENTEKST.txt i xml_output\brev
>>> Metadata opprettet: xml_output\brev\metadata\B1844-1871ht_METADATA.txt


2 xml_src\brev\B1871-1879ht.xml
** LOGG Filen har 476 tekster **
Henter metadata...
Konverterer teksten...
> 5624 uønskede tagger
> 13 unike unicode-tegn
Lagrer...
>> Opprettet xml_output\brev\B1871-1879ht_RENTEKST.txt i xml_output\brev
>>> Metadata opprettet: xml_output\brev\metadata\B1871-1879ht_METADATA.txt


3 xml_src\brev\B1880-1889ht.xml
** LOGG Filen har 628 tekster **
Henter metadata...
Konverterer teksten...
> 6178 uønskede tagger
> 12 unike unicode-tegn
Lagrer...
>> Opprettet xml_output\brev\B1880-1889ht_RENTEKST.txt i xml_output\brev
>>> Metadata opprettet: xml_output\brev\metadata\B1880-1889ht_METADATA.txt


4 xml_src\brev\B1890-1905ht.xml
** LOGG Filen har 972 tekster **
Hent

> 1 HTML-kommentarer
> 6 unike unicode-tegn
Lagrer...
>> Opprettet xml_output\drama\Br\Br4262I11_RENTEKST.txt i xml_output\drama\Br
>>> Metadata opprettet: xml_output\drama\Br\metadata\Br4262I11_METADATA.txt


30 xml_src\drama\Br\Br4262I12.xml
Henter metadata...
Konverterer teksten...
> 8728 uønskede tagger
> 1 HTML-kommentarer
> 6 unike unicode-tegn
Lagrer...
>> Opprettet xml_output\drama\Br\Br4262I12_RENTEKST.txt i xml_output\drama\Br
>>> Metadata opprettet: xml_output\drama\Br\metadata\Br4262I12_METADATA.txt


31 xml_src\drama\Br\Br66.xml
Henter metadata...
Konverterer teksten...
> 8690 uønskede tagger
> 6 unike unicode-tegn
Lagrer...
>> Opprettet xml_output\drama\Br\Br66_RENTEKST.txt i xml_output\drama\Br
>>> Metadata opprettet: xml_output\drama\Br\metadata\Br66_METADATA.txt


32 xml_src\drama\Br\Br66a.xml
Henter metadata...
Konverterer teksten...
> 8692 uønskede tagger
> 6 unike unicode-tegn
Lagrer...
>> Opprettet xml_output\drama\Br\Br66a_RENTEKST.txt i xml_output\drama\Br
>>> Me

** LOGG Filen har 4 tekster **
Henter metadata...
Konverterer teksten...
> 4182 uønskede tagger
> 20 superlineære tallnoter
> 2 HTML-kommentarer
> 7 unike unicode-tegn
Lagrer...
>> Opprettet xml_output\drama\C1\C14936_NBO_RENTEKST.txt i xml_output\drama\C1
>>> Metadata opprettet: xml_output\drama\C1\metadata\C14936_NBO_METADATA.txt


62 xml_src\drama\C1\C14936_NBOgammel.xml
** LOGG Filen har 4 tekster **
Henter metadata...
Konverterer teksten...
> 4183 uønskede tagger
> 20 superlineære tallnoter
> 7 unike unicode-tegn
Lagrer...
>> Opprettet xml_output\drama\C1\C14936_NBOgammel_RENTEKST.txt i xml_output\drama\C1
>>> Metadata opprettet: xml_output\drama\C1\metadata\C14936_NBOgammel_METADATA.txt


63 xml_src\drama\C1\C150.xml
Henter metadata...
Konverterer teksten...
> 3462 uønskede tagger
> 6 unike unicode-tegn
Lagrer...
>> Opprettet xml_output\drama\C1\C150_RENTEKST.txt i xml_output\drama\C1
>>> Metadata opprettet: xml_output\drama\C1\metadata\C150_METADATA.txt


64 xml_src\drama\C1\C1h

...

> 7 unike unicode-tegn
Lagrer...
>> Opprettet xml_output\drama\LE\LEFU_RENTEKST.txt i xml_output\drama\LE
>>> Metadata opprettet: xml_output\drama\LE\metadata\LEFU_METADATA.txt


337 xml_src\drama\LE\LEht.xml
Henter metadata...
Konverterer teksten...
> 718 uønskede tagger
> 7 unike unicode-tegn
Lagrer...
>> Opprettet xml_output\drama\LE\LEht_RENTEKST.txt i xml_output\drama\LE
>>> Metadata opprettet: xml_output\drama\LE\metadata\LEht_METADATA.txt


339 xml_src\drama\LE\LELondon.xml
Henter metadata...
Konverterer teksten...
> 4790 uønskede tagger
> 7 unike unicode-tegn
Lagrer...
>> Opprettet xml_output\drama\LE\LELondon_RENTEKST.txt i xml_output\drama\LE
>>> Metadata opprettet: xml_output\drama\LE\metadata\LELondon_METADATA.txt


340 xml_src\drama\LE\LE_a_privat.xml
Henter metadata...
Konverterer teksten...
> 198 uønskede tagger
> 1 HTML-kommentarer
> 4 unike unicode-tegn
Lagrer...
>> Opprettet xml_output\drama\LE\LE_a_privat_RENTEKST.txt i xml_output\drama\LE
>>> Metadata opprettet: xml

> 6 unike unicode-tegn
Lagrer...
>> Opprettet xml_output\drama\O1\O1344_Hemming_RENTEKST.txt i xml_output\drama\O1
>>> Metadata opprettet: xml_output\drama\O1\metadata\O1344_Hemming_METADATA.txt


368 xml_src\drama\O1\O1344_Ingeborg.xml
Henter metadata...
Konverterer teksten...
> 557 uønskede tagger
> 6 unike unicode-tegn
Lagrer...
>> Opprettet xml_output\drama\O1\O1344_Ingeborg_RENTEKST.txt i xml_output\drama\O1
>>> Metadata opprettet: xml_output\drama\O1\metadata\O1344_Ingeborg_METADATA.txt


369 xml_src\drama\O1\O1344_MaendKvinder1.xml
Henter metadata...
Konverterer teksten...
> 144 uønskede tagger
> 6 unike unicode-tegn
Lagrer...
>> Opprettet xml_output\drama\O1\O1344_MaendKvinder1_RENTEKST.txt i xml_output\drama\O1
>>> Metadata opprettet: xml_output\drama\O1\metadata\O1344_MaendKvinder1_METADATA.txt


370 xml_src\drama\O1\O1344_MaendKvinder2.xml
Henter metadata...
Konverterer teksten...
> 138 uønskede tagger
> 6 unike unicode-tegn
Lagrer...
>> Opprettet xml_output\drama\O1\O1344_M

Henter metadata...
Konverterer teksten...
> 8302 uønskede tagger
> 7 unike unicode-tegn
Lagrer...
>> Opprettet xml_output\drama\PG\PG91_RENTEKST.txt i xml_output\drama\PG
>>> Metadata opprettet: xml_output\drama\PG\metadata\PG91_METADATA.txt


400 xml_src\drama\PG\PG93.xml
Henter metadata...
Konverterer teksten...
> 8301 uønskede tagger
> 7 unike unicode-tegn
Lagrer...
>> Opprettet xml_output\drama\PG\PG93_RENTEKST.txt i xml_output\drama\PG
>>> Metadata opprettet: xml_output\drama\PG\metadata\PG93_METADATA.txt


401 xml_src\drama\PG\PG96.xml
Henter metadata...
Konverterer teksten...
> 8301 uønskede tagger
> 7 unike unicode-tegn
Lagrer...
>> Opprettet xml_output\drama\PG\PG96_RENTEKST.txt i xml_output\drama\PG
>>> Metadata opprettet: xml_output\drama\PG\metadata\PG96_METADATA.txt


402 xml_src\drama\PG\PG99.xml
Henter metadata...
Konverterer teksten...
> 8230 uønskede tagger
> 7 unike unicode-tegn
Lagrer...
>> Opprettet xml_output\drama\PG\PG99_RENTEKST.txt i xml_output\drama\PG
>>> Met

Henter metadata...
Konverterer teksten...
> 4626 uønskede tagger
> 7 unike unicode-tegn
Lagrer...
>> Opprettet xml_output\drama\Sa\Sa872s_RENTEKST.txt i xml_output\drama\Sa
>>> Metadata opprettet: xml_output\drama\Sa\metadata\Sa872s_METADATA.txt


433 xml_src\drama\Sa\SahBI.xml
Henter metadata...
Konverterer teksten...
> 3852 uønskede tagger
> 6 unike unicode-tegn
Lagrer...
>> Opprettet xml_output\drama\Sa\SahBI_RENTEKST.txt i xml_output\drama\Sa
>>> Metadata opprettet: xml_output\drama\Sa\metadata\SahBI_METADATA.txt


434 xml_src\drama\Sa\Saht.xml
** LOGG Filen har 2 tekster **
Henter metadata...
Konverterer teksten...
> 643 uønskede tagger
> 1 HTML-kommentarer
> 9 unike unicode-tegn
Lagrer...
>> Opprettet xml_output\drama\Sa\Saht_RENTEKST.txt i xml_output\drama\Sa
>>> Metadata opprettet: xml_output\drama\Sa\metadata\Saht_METADATA.txt


435 xml_src\drama\Sa\Saht_gml.xml
** LOGG Filen har 2 tekster **
Henter metadata...
Konverterer teksten...
> 643 uønskede tagger
> 9 unike unicode-teg

Henter metadata...
Konverterer teksten...
> 6250 uønskede tagger
> 6 unike unicode-tegn
Lagrer...
>> Opprettet xml_output\drama\SS\SS93_RENTEKST.txt i xml_output\drama\SS
>>> Metadata opprettet: xml_output\drama\SS\metadata\SS93_METADATA.txt


462 xml_src\drama\SS\SSFU.xml
Henter metadata...
Konverterer teksten...
> 5990 uønskede tagger
> 6 unike unicode-tegn
Lagrer...
>> Opprettet xml_output\drama\SS\SSFU_RENTEKST.txt i xml_output\drama\SS
>>> Metadata opprettet: xml_output\drama\SS\metadata\SSFU_METADATA.txt


463 xml_src\drama\SS\SSht.xml
Henter metadata...
Konverterer teksten...
> 1075 uønskede tagger
> 7 unike unicode-tegn
Lagrer...
>> Opprettet xml_output\drama\SS\SSht_RENTEKST.txt i xml_output\drama\SS
>>> Metadata opprettet: xml_output\drama\SS\metadata\SSht_METADATA.txt


465 xml_src\drama\Sv\Sv4926.xml
Henter metadata...
Konverterer teksten...
> 751 uønskede tagger
> 6 unike unicode-tegn
Lagrer...
>> Opprettet xml_output\drama\Sv\Sv4926_RENTEKST.txt i xml_output\drama\Sv
>>> 

## Logg

In [14]:
print("Behandlet",str(percA)+"% av",len(paths),"filer med",y,"lagringsfeil.")
if errors:
    errors

Behandlet 100.0% av 493 filer med 0 lagringsfeil.


# Ferdig!
De rene tekstfilene skal nå finnes under .../xml_output/*, samt metadata i tilhørende mappe (om createMetadata == 1). Med mindre du fikk feil i scriptet over. Eller om noe annet gikk galt.
___